# Sparse Array Structure
Sparse arrays are in compressed column format according to the doumentation
    https://docs.julialang.org/en/v1/stdlib/SparseArrays/#man-csc
The entries are directly accessible and editable.  Not surprisingly the documentation say adding entries is slow!

In [59]:
using SparseArrays, LinearAlgebra
(m,n)=(12,13)
A=sprandn(m,n,0.1)

12×13 SparseMatrixCSC{Float64,Int64} with 21 stored entries:
  [1 ,  1]  =  0.570908
  [9 ,  1]  =  -0.00993403
  [2 ,  3]  =  -0.689925
  [4 ,  3]  =  -2.39669
  [9 ,  3]  =  0.605451
  [2 ,  4]  =  -0.798289
  [5 ,  4]  =  0.0108901
  [6 ,  4]  =  0.00705011
  [7 ,  4]  =  -0.615599
  [8 ,  4]  =  -0.433664
  [9 ,  5]  =  0.0202746
  [2 ,  7]  =  0.648151
  [4 ,  7]  =  -0.465501
  [1 ,  9]  =  -1.49707
  [9 , 10]  =  0.155165
  [1 , 11]  =  0.115463
  [7 , 11]  =  0.266804
  [10, 12]  =  -0.0848329
  [12, 12]  =  -0.133205
  [10, 13]  =  -0.633766
  [12, 13]  =  1.20935

## Sparse Array Acessors

****

The accessors for the stored CSC structure of a sparse $m\times n$ array are 

* A.m 
    * Number of Rows 
* A.n
    * Number of columns
* A.colptr
    * pointers to start of columns
* A.rowval
    * Row Indices of stored values
* A.nzval
    * Stored values
    
They should be performant. 

### Demos 

* The dimensions probably need no explanation
    * Structural
* The values probably need little explanation. Note the integer values in row val have been automatically promoted in the display
    * Values
* Column Structure may need more interpretation and explanation
    * column pointers give the start (in the value lists above) of each column.    

In [23]:
(A.m,A.n)

(12, 13)

In [30]:
[A.rowval A.nzval]

18×2 Array{Float64,2}:
  1.0  -1.35688 
  3.0   1.45905 
  4.0  -0.19126 
  5.0  -0.68639 
 10.0  -0.939249
 11.0  -1.81431 
  7.0  -0.972066
  3.0   1.33456 
  9.0  -0.218103
 11.0  -0.195553
  4.0  -0.264478
  4.0  -2.18732 
  5.0  -1.05255 
  6.0  -0.550504
  1.0   0.368684
 12.0   0.459757
  6.0   0.386804
  8.0   2.05498 

In [32]:
A.colptr

14-element Array{Int64,1}:
  1
  4
  5
  7
  8
 10
 11
 12
 15
 17
 17
 17
 19
 19

In [57]:
c = 3
display(A[:,c])
p = A.colptr
r = A.rowval
z = A.nzval
range = (p[c]):(p[c+1]-1)
(r[range],z[range])

12-element SparseVector{Float64,Int64} with 2 stored entries:
  [10]  =  -0.939249
  [11]  =  -1.81431

([10, 11], [-0.939249, -1.81431])

# Sparse Matrix Operations

Timings maybe take a little bit of getting used to.  They make a lot more sense if you remember that 
Column based operations are faster since the underlying storage is by column.

In [102]:
m=10^5;
@time A=sprand(m,m,0.005)
@time B=copy(A)
@time B=copy(A)'
@time B=copy(A')
@time A=A';

  1.539028 seconds (20 allocations: 1.120 GiB, 14.74% gc time)
  0.176234 seconds (11 allocations: 763.597 MiB, 0.74% gc time)
  0.353601 seconds (12 allocations: 763.597 MiB, 24.67% gc time)
  0.904620 seconds (13 allocations: 763.597 MiB, 1.40% gc time)
  0.000001 seconds (5 allocations: 176 bytes)


# Sparse Matrix Dense Vector Operations
Remember, column based operations are faster 

In [113]:
s=16;
@time V=rand(m,s)
@time A*V
@time A'*V
@time B*V
@time B'*V;

  0.003088 seconds (6 allocations: 12.207 MiB)
  0.798068 seconds (7 allocations: 12.207 MiB, 0.23% gc time)
  0.850260 seconds (6 allocations: 12.207 MiB)
  0.855620 seconds (6 allocations: 12.207 MiB, 0.08% gc time)
  0.760032 seconds (8 allocations: 12.207 MiB)


# Sparse Matrix Sparse Matrix Operations
Should have similar patterns.

In [115]:
@time V=sprand(m,s,0.001)
@time A*V
@time A'*V
@time B*V
@time B'*V;

  0.000198 seconds (19 allocations: 91.156 KiB)
  1.001180 seconds (29 allocations: 791.826 MiB, 8.30% gc time)
  0.061604 seconds (21 allocations: 28.230 MiB, 42.48% gc time)
  0.036416 seconds (21 allocations: 28.230 MiB, 1.77% gc time)
  0.899411 seconds (30 allocations: 791.826 MiB, 0.17% gc time)


# Slightly Counter Intuitive Timings 
Two ways of doing the same thing. 

In [12]:
using SparseArrays, LinearAlgebra
(m,s)=(10^4,16);
@time A=sprand(m,m,0.005)
@time V=rand(m,s)
@time print(1); P1 = A*V; P1=P1'
@time print(2); P2 = V'*A';
@time print(3); norm(P1-P2)

  0.035961 seconds (22 allocations: 19.307 MiB, 14.39% gc time)
  0.000814 seconds (6 allocations: 1.221 MiB)
1  0.000031 seconds (14 allocations: 400 bytes)
2  0.000074 seconds (15 allocations: 800 bytes)
3  0.000240 seconds (157 allocations: 10.141 KiB)


5.842931536447421e-13

In [18]:
using SparseArrays, LinearAlgebra
(m,s)=(10^4,16);
@time A=sprand(m,m,0.005)
@time V=sprand(m,s,0.05)
@time print(1); P1 = A*V; P1=P1'
@time print(2); P2 = V'*A';
@time print(3); norm(P1-P2)

  0.030227 seconds (20 allocations: 11.672 MiB, 8.07% gc time)
  0.000385 seconds (21 allocations: 453.891 KiB)
1  0.000035 seconds (14 allocations: 400 bytes)
2  0.000057 seconds (14 allocations: 400 bytes)
3  0.000043 seconds (14 allocations: 400 bytes)


0.0

# Sparse Vectors
Sparse vectors are the same idea but simpler. Under some circumstances we might want to think of matrices as collections of rows! 

In [23]:
m=10^2
v=sprand(m,0.01)

100-element SparseVector{Float64,Int64} with 3 stored entries:
  [17 ]  =  0.227517
  [43 ]  =  0.252094
  [56 ]  =  0.739746

In [27]:
display(v.n)
display((v.nzind,v.nzval))

100

([17, 43, 56], [0.227517, 0.252094, 0.739746])